#### 1. Install Packages

In [1]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install pypdf
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [2]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 15.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 75.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/6

In [3]:
import openai
from langchain.document_loaders import PyPDFLoader, TextLoader, DirectoryLoader
# converting documents into smaller chunks of token
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Embeddings
from langchain.embeddings import OpenAIEmbeddings
# Storing and creating a persitance vectorized form of text
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [4]:
import warnings
warnings.filterwarnings('ignore')

#### Read the data from directory and create a document

In [5]:
doc_loader = PyPDFLoader('/kaggle/input/humanresourcehr-policy/humanresourcespolicy.pdf')
documents = doc_loader.load()

In [6]:
documents

[Document(metadata={'source': '/kaggle/input/humanresourcehr-policy/humanresourcespolicy.pdf', 'page': 0, 'page_label': '1'}, page_content='The Nestlé Human Resources Policy\n© October 2002, Nestec Ltd., Human Resources Department\nConcept and design: Nestec Ltd., B-COM , Corporate Identity and Design, Vevey, Switzerland\nPrinted by Neidhart + Schön AG, Zurich, Switzerland'),
 Document(metadata={'source': '/kaggle/input/humanresourcehr-policy/humanresourcespolicy.pdf', 'page': 1, 'page_label': '2'}, page_content='2 The Nestlé human Resources Policy\n4 A shared Responsibility\n5 Dealing with People\n6 Joining Nestlé\n8 Employment at Nestlé\n9 Work/Life Balance\n10 Remuneration\n11 Professional Development\n14 Industrial Relations\n15 HR Organisation\nThe Nestlé Human Resources Policy\nTable of Contents'),
 Document(metadata={'source': '/kaggle/input/humanresourcehr-policy/humanresourcespolicy.pdf', 'page': 2, 'page_label': '3'}, page_content='The Nestlé\nHuman Resources Policy'),
 Docum

#### TextSplitter - RecurssiveTextSplitter

In [7]:
# split documents into smaller chunk and maximum chunk size = 100.
# 20 characters of overlap between chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 20)
texts = text_splitter.split_documents(documents)

In [8]:
texts

[Document(metadata={'source': '/kaggle/input/humanresourcehr-policy/humanresourcespolicy.pdf', 'page': 0, 'page_label': '1'}, page_content='The Nestlé Human Resources Policy\n© October 2002, Nestec Ltd., Human Resources Department\nConcept and design: Nestec Ltd., B-COM , Corporate Identity and Design, Vevey, Switzerland'),
 Document(metadata={'source': '/kaggle/input/humanresourcehr-policy/humanresourcespolicy.pdf', 'page': 0, 'page_label': '1'}, page_content='Printed by Neidhart + Schön AG, Zurich, Switzerland'),
 Document(metadata={'source': '/kaggle/input/humanresourcehr-policy/humanresourcespolicy.pdf', 'page': 1, 'page_label': '2'}, page_content='2 The Nestlé human Resources Policy\n4 A shared Responsibility\n5 Dealing with People\n6 Joining Nestlé\n8 Employment at Nestlé\n9 Work/Life Balance\n10 Remuneration\n11 Professional Development'),
 Document(metadata={'source': '/kaggle/input/humanresourcehr-policy/humanresourcespolicy.pdf', 'page': 1, 'page_label': '2'}, page_content='1

#### Create Embeddings and store vector representation of inputs

In [9]:
api_key=""

In [10]:
openai_embeddings = OpenAIEmbeddings(openai_api_key = api_key)

In [11]:
vectordb = Chroma.from_documents(
    documents = texts,
    embedding = openai_embeddings,
    persist_directory = "/kaggle/working/vector_store/"
)
vectordb.persist()

#### Create OpenAI Retrieval Model

In [13]:
from langchain.chat_models import ChatOpenAI

# build llm
llm = ChatOpenAI(model_name = "gpt-3.5-turbo",
                openai_api_key = api_key,
                temperature = 1.0,
                max_tokens = 1000)

In [14]:
Retrieval_Chain = RetrievalQA.from_chain_type(llm,
                           retriever = vectordb.as_retriever(),
                           return_source_documents = True)

### Get QA Response from Retrieval QA

In [15]:
chat_history = []
print("I am your virtualHR @ Nestle. Ask me anything.")
while True:
    prompt = input("User: ")
    if prompt.lower() == 'quit':
        break
    if prompt.strip == "":
        continue
    reply = Retrieval_Chain(prompt)
    print(f"\n HR Response : ")
    print(reply['result'])

I am your virtualHR @ Nestle. Ask me anything.


User:  how is the worklife balance in your company



 HR Response : 
At Nestlé, the company believes in maintaining a good balance between an employee's private and professional life. This balance is seen as crucial for enhancing productivity, satisfaction, loyalty, and positively reflecting on the company's reputation. Nestlé encourages flexible working arrangements to support work-life balance, and employees are encouraged to communicate their objectives and expectations openly.


User:  quit
